## Intro

This is inspired by 
Article (likas2001probability) Likas, A. Probability density estimation using artificial neural networks Computer physics communications, Elsevier, 2001, 135, 167-175

But rather than estimating the working with a network, we will instead work with its derivitive.
This will let us replace their integration with a derivative.

Note that this method only works for compact supports



They use the PDF is given by $$p_h(x,p) = \dfrac{h(x,p)}{\int_S h(z,p) dz}$$
and in their case $h=N(x,p)$  a neural network with weight and bias parameters $p$.
Where $S$ is a compact support. (That means bounded)


But if instead we say $h=\frac{\partial N(x,p)}{\partial x}$,

then $$p_h(x,p) = \dfrac{h(x,p)}{\int_S h(z,p)}=\dfrac{\frac{\partial N(x,p)}{\partial x}}{N(max(S),p) - N(min(S), p)}$$

The denominator is ofcourse more complex for non-1D values of S.


The loss function given is the negative log-likelihood of the set of training samples $X$
$$L(p) = -\sum_{\forall x \in X} ln(h(x,p))  + |X| ln(\int_S h(z,p) dx)$$

Which befomes:

$$L(p) = -\sum_{\forall x \in X} log(\frac{\partial N(x,p)}{\partial x})  + |X|(ln(N(max(S),p)-N(min(S),p)) dx$$

In [1]:
using StatsBase
using Distributions

In [2]:
using TensorFlow
using MLDataUtils

In [3]:
using DensityEstimationML

In [4]:
immutable NeuralDensityEstimator
    sess::Session
    
    #Network nodes
    optimizer::Tensor
    conditioner::Tensor
    t::Tensor
    pdf::Tensor
end

In [125]:
loglikelihood(est,[1,2])

LoadError: [91mNode loglikelyhood not found in graph[39m

In [177]:
function Distributions.pdf(est::NeuralDensityEstimator, t::Real)
    ts = reshape([t], (1,1))
    pdf(est, ts) |> first
end

function Distributions.pdf(est::NeuralDensityEstimator, ts::AbstractVector)
    gr = est.sess.graph
    run(est.sess, est.pdf, Dict(est.t=>ts')) |> vec
end

function Distributions.loglikelihood(est::NeuralDensityEstimator, ts::AbstractVector)
    gr = est.sess.graph
    run(est.sess, gr["loglikelihood"], Dict(est.t=>ts')) |> vec
end


In [178]:
function StatsBase.fit!(estimator::NeuralDensityEstimator, observations;
    epochs = 20)
    
    gr = estimator.sess.graph
    for ii in 1:epochs
        loss_o = run(estimator.sess, 
            [gr["ysmin"],gr["ysmax"],gr["loglikelihood"], gr["working_loss"],
                estimator.optimizer],
            Dict(estimator.t=>observations'))
        ii % 100 == 1 && println("Epoch $ii: loss: $(loss_o)")
    end
    estimator
end

In [179]:
function NeuralDensityEstimator(prob_layer_sizes, support)
    sess = Session(Graph())
    @tf begin
        t = placeholder(Float32, shape=[1, -1])
        smin = constant(reshape([minimum(support)],(1,1)))
        smax = constant(reshape([maximum(support)],(1,1)))
        
        layer_sizes= [1; prob_layer_sizes; 1]
        
        network_fun_stack = Function[Base.identity]       
       
        for ii in 2:length(layer_sizes)
            below_size = layer_sizes[ii-1]
            above_size = layer_sizes[ii]
                       
            Wii = get_variable("W_$ii", [above_size, below_size], Float32)
            Wii2  = Ops.mul(Wii, Wii; name = "W_$(ii)_squared")
            act_fun = if ii!=length(layer_sizes)
                bii = get_variable("b_$ii", [above_size, 1], Float32)
                z -> nn.sigmoid(Wii2*z .+ bii)
            else
                z-> exp(Wii2*z)
            end
            push!(network_fun_stack, z->act_fun(network_fun_stack[ii-1](z)))
        end
        
        network = network_fun_stack[end]

        
        ysmin = TensorFlow.identity(network(smin))
        ysmax = TensorFlow.identity(network(smax))
        yt = network(t)
        
        denominator = (ysmax-ysmin) #area
        numerator = gradients(yt,t)
        pdf =numerator/denominator
        
        
        n_points = TensorFlow.shape(t)[2]
        loglikelihood = reduce_sum(log(numerator)) - n_points.*log(denominator)
        
        area_loss = (1f0.-denominator)^2
        working_loss = -1*loglikelihood + 0.1*area_loss
        
        optimizer = train.minimize(train.AdamOptimizer(), working_loss)
        
        
        # Conditioning
        # Make sure that ysmin~=1, and ysmax~=2
        condition_loss = (1f0 - ysmin)^2 + (2f0 - ysmax)^2
        condition_optimiser = train.minimize(train.AdamOptimizer(;name="adam_cond"), condition_loss)
    end
    
    run(sess, global_variables_initializer())
    
    NeuralDensityEstimator(sess, optimizer, condition_optimiser, t, pdf)
end

NeuralDensityEstimator

In [180]:
"""
    condition(est::NeuralDensityEstimator tol = 1e-15, max_epochs=2_000)
    
"Conditions" the neural density estimate so the support extrema are mapped to 1. and 2.
This improves training by adjusting the area the network has the learn over

"""
function condition!(est::NeuralDensityEstimator, tol = 1e-15, max_epochs=2_000)
    gr = est.sess.graph
    for ii in 1:2_000
        _, ysmin, ysmax, condition_loss = run(est.sess, [est.conditioner, gr["ysmin"],gr["ysmax"], gr["condition_loss"]])
        ii % 50 == 1 && @show (ii, ysmin, ysmax, condition_loss)
        if condition_loss[1] < 1e-15
            break
        end
    end
end


condition!

In [181]:
dataset =GenerateDatasets.Likas1()
data = rand(dataset)
@show loglikelihood(dataset, data)
est = NeuralDensityEstimator([64,64], approximate_support(dataset))


loglikelihood(dataset, data) = -10480.494472314513


2017-09-12 18:26:36.130845: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1030] Creating TensorFlow device (/gpu:0) -> (device: 0, name: GeForce GTX TITAN X, pci bus id: 0000:01:00.0)


NeuralDensityEstimator(Session(Ptr{Void} @0x00007fc39ce49a30), <Tensor Group:1 shape=unknown dtype=Any>, <Tensor Group_2:1 shape=unknown dtype=Any>, <Tensor t:1 shape=(1, ?) dtype=Float32>, <Tensor pdf:1 shape=unknown dtype=Float64>)

In [182]:
condition!(est)
println("Conditioning Done")
fit!(est, data; epochs=20_000)

(ii, ysmin, ysmax, condition_loss) = (1, [1.00289], [1.00289], [0.994246])
(ii, ysmin, ysmax, condition_loss) = (51, [1.16893], [1.16937], [0.718487])
(ii, ysmin, ysmax, condition_loss) = (101, [1.51794], [1.54766], [0.472872])
(ii, ysmin, ysmax, condition_loss) = (151, [1.49805], [1.61684], [0.39486])
(ii, ysmin, ysmax, condition_loss) = (201, [1.46297], [1.72065], [0.292374])
(ii, ysmin, ysmax, condition_loss) = (251, [1.42254], [1.79785], [0.21941])
(ii, ysmin, ysmax, condition_loss) = (301, [1.38931], [1.84119], [0.176787])
(ii, ysmin, ysmax, condition_loss) = (351, [1.36476], [1.86682], [0.150785])
(ii, ysmin, ysmax, condition_loss) = (401, [1.34582], [1.88392], [0.13307])
(ii, ysmin, ysmax, condition_loss) = (451, [1.33034], [1.89648], [0.119844])
(ii, ysmin, ysmax, condition_loss) = (501, [1.31708], [1.90636], [0.109307])
(ii, ysmin, ysmax, condition_loss) = (551, [1.30531], [1.91451], [0.100523])
(ii, ysmin, ysmax, condition_loss) = (601, [1.2946], [1.92146], [0.0929577])
(ii, 

Epoch 6701: loss: Any[[1.00794], [5.83183], [-10777.2], [10778.7], nothing]
Epoch 6801: loss: Any[[1.00746], [5.76488], [-10774.0], [10775.4], nothing]
Epoch 6901: loss: Any[[1.00699], [5.6986], [-10771.1], [10772.4], nothing]
Epoch 7001: loss: Any[[1.00654], [5.62821], [-10768.4], [10769.7], nothing]
Epoch 7101: loss: Any[[1.00611], [5.55005], [-10765.8], [10767.1], nothing]
Epoch 7201: loss: Any[[1.0057], [5.46446], [-10763.5], [10764.7], nothing]
Epoch 7301: loss: Any[[1.00532], [5.37491], [-10761.3], [10762.5], nothing]
Epoch 7401: loss: Any[[1.00496], [5.28522], [-10759.3], [10760.4], nothing]
Epoch 7501: loss: Any[[1.00463], [5.19773], [-10757.4], [10758.4], nothing]
Epoch 7601: loss: Any[[1.00432], [5.11319], [-10755.5], [10756.5], nothing]
Epoch 7701: loss: Any[[1.00402], [5.03191], [-10753.8], [10754.7], nothing]
Epoch 7801: loss: Any[[1.00375], [4.95356], [-10752.0], [10752.9], nothing]
Epoch 7901: loss: Any[[1.0035], [4.87781], [-10750.3], [10751.2], nothing]
Epoch 8001: los

Epoch 17501: loss: Any[[1.00001], [2.25562], [-10567.6], [10567.6], nothing]
Epoch 17601: loss: Any[[1.00001], [2.24834], [-10566.9], [10566.9], nothing]
Epoch 17701: loss: Any[[1.00001], [2.24113], [-10566.2], [10566.2], nothing]
Epoch 17801: loss: Any[[1.0], [2.23417], [-10565.6], [10565.6], nothing]
Epoch 17901: loss: Any[[1.0], [2.22741], [-10564.9], [10564.9], nothing]
Epoch 18001: loss: Any[[1.0], [2.2208], [-10564.3], [10564.3], nothing]
Epoch 18101: loss: Any[[1.0], [2.21452], [-10563.7], [10563.7], nothing]
Epoch 18201: loss: Any[[1.0], [2.20809], [-10563.0], [10563.0], nothing]
Epoch 18301: loss: Any[[1.0], [2.20184], [-10562.4], [10562.4], nothing]
Epoch 18401: loss: Any[[1.0], [2.19572], [-10561.8], [10561.8], nothing]
Epoch 18501: loss: Any[[1.0], [2.1897], [-10561.2], [10561.2], nothing]
Epoch 18601: loss: Any[[1.0], [2.18392], [-10560.6], [10560.6], nothing]
Epoch 18701: loss: Any[[1.0], [2.1783], [-10560.1], [10560.1], nothing]
Epoch 18801: loss: Any[[1.0], [2.17286], [

NeuralDensityEstimator(Session(Ptr{Void} @0x00007fc39ce49a30), <Tensor Group:1 shape=unknown dtype=Any>, <Tensor Group_2:1 shape=unknown dtype=Any>, <Tensor t:1 shape=(1, ?) dtype=Float32>, <Tensor pdf:1 shape=unknown dtype=Float64>)

In [183]:
function demonstration_plot(est, dataset, data=rand(dataset), args...; kwargs...)
    X = minimum(approximate_support(dataset)) : 0.01 : maximum(approximate_support(dataset))
    println("True loglikelihood      = $(loglikelihood(dataset, data))")
    println("Estimated loglikelihood = $(loglikelihood(est, data))")
    estimated_loglikelihood = loglikelihood(dataset, data)
    plot([X], [pdf(est,X), data],
        #xlims= approximate_support(dataset),
        xlims= (-10,10),
        seriestype = [:path :histogram],
        layout=(2,1),
        legend=false,
        nbins=[1  length(X)÷10],
        args...; kwargs...
    )
end



demonstration_plot (generic function with 2 methods)

In [184]:
demonstration_plot(est, dataset, data)

True loglikelihood      = -10480.494472314513
Estimated loglikelihood = [-10553.3]


<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -10 
 
 
 -5 
 
 
 0 
 
 
 5 
 
 
 10 
 
 
 0.05 
 
 
 0.10 
 
 
 0.15 
 
 
 0.20 
 
<polyline clip-path="url(#clip02)" style="stroke:#009af9; stroke-width:1; stroke-opacity:1; fill:none" points="
 -15.3386,175.813 -15.0678,175.813 -14.7971,175.813 -14.5263,175.813 -14.2556,175.813 -13.9848,175.813 -13.7141,175.813 -13.4433,175.813 -13.1726,175.813 -12.9018,175.813 
 -12.6311,175.813 -12.3603,175.813 -12.0896,175.813 -11.8188,175.813 -11.5481,175.813 -11.2773,175.813 -11.0066,175.813 -10.7358,175.813 -10.4651,175.813 -10.1943,175.813 
 -9.92356,175.813 -9.6528,175.813 -9.38205,175.813 -9.1113,175.813 -8.84055,175.813 -8.5698,175.813 -8.29905,175.813 -8.02829,175.813 -7.75754,175.813 -7.48679,175.813 
 -7.21604,175.813 -6.94529,175.813 -6.67454,175.813 -6.40378,175.813 -6.13303,175.813 -5.86228,175.813 -5.59153,175.813 -5.32078,175.813 -5.05003,175.813 -4.77927,175.813 
 -4.50852,175.813 -4.23777,175.813 -3.96702,175.813 -3.69627,175.813 -3.42552,175.813 -3.15476,175.813 -2.88401,175.813 -2.61326,175.813 -2.34251,175.813 -2.07176,175.813 
 -1.80101,175.813 -1.53026,175.813 -1.2595,175.813 -0.988752,175.813 -0.718,175.813 -0.447249,175.813 -0.176497,175.813 0.0942546,175.813 0.365006,175.813 0.635758,175.813 
 0.906509,175.813 1.17726,175.813 1.44801,175.813 1.71876,175.813 1.98952,175.813 2.26027,175.813 2.53102,175.813 2.80177,175.813 3.07252,175.813 3.34327,175.813 
 3.61403,175.813 3.88478,175.813 4.15553,175.813 4.42628,175.813 4.69703,175.813 4.96778,175.813 5.23854,175.813 5.50929,175.813 5.78004,175.813 6.05079,175.813 
 6.32154,175.813 6.59229,175.813 6.86304,175.813 7.1338,175.813 7.40455,175.813 7.6753,175.813 7.94605,175.813 8.2168,175.813 8.48755,175.813 8.75831,175.813 
 9.02906,175.813 9.29981,175.813 9.57056,175.813 9.84131,175.813 10.1121,175.813 10.3828,175.813 10.6536,175.813 10.9243,175.813 11.1951,175.813 11.4658,175.813 
 11.7366,175.813 12.0073,175.813 12.2781,175.813 12.5488,175.813 12.8196,175.813 13.0903,175.813 13.3611,175.813 13.6318,175.813 13.9026,175.813 14.1733,175.813 
 14.4441,175.813 14.7148,175.813 14.9856,175.813 15.2563,175.813 15.5271,175.813 15.7978,175.813 16.0686,175.813 16.3394,175.813 16.6101,175.813 16.8809,175.813 
 17.1516,175.813 17.4224,175.813 17.6931,175.813 17.9639,175.813 18.2346,175.813 18.5054,175.813 18.7761,175.813 19.0469,175.813 19.3176,175.813 19.5884,175.813 
 19.8591,175.813 20.1299,175.813 20.4006,175.813 20.6714,175.813 20.9421,175.813 21.2129,175.813 21.4836,175.813 21.7544,175.812 22.0251,175.812 22.2959,175.812 
 22.5666,175.812 22.8374,175.812 23.1081,175.812 23.3789,175.812 23.6496,175.812 23.9204,175.812 24.1911,175.812 24.4619,175.812 24.7327,175.812 25.0034,175.812 
 25.2742,175.812 25.5449,175.812 25.8157,175.812 26.0864,175.812 26.3572,175.812 26.6279,175.812 26.8987,175.812 27.1694,175.812 27.4402,175.812 27.7109,175.812 
 27.9817,175.812 28.2524,175.812 28.5232,175.812 28.7939,175.812 29.0647,175.812 29.3354,175.812 29.6062,175.812 29.8769,175.812 30.1477,175.812 30.4184,175.811 
 30.6892,175.811 30.9599,175.811 31.2307,175.811 31.5014,175.811 31.7722,175.811 32.0429,175.811 32.3137,175.811 32.5844,175.811 32.8552,175.811 33.126,175.811 
 33.3967,175.811 33.6675,175.811 33.9382,175.811 34.209,175.811 34.4797,175.811 34.7505,175.811 35.0212,175.81 35.292,175.81 35.5627,175.81 35.8335,175.81 
 36.1042,175.81 36.375,175.81 36.6457,175.81 36.9165,175.81 37.1872,175.81 37.458,175.81 37.7287,175.81 37.9995,175.809 38.2702,175.809 38.541,175.809 
 38.8117,175.809 39.0825,175.809 39.3532,175.809 39.624,175.809 39.8947,175.809 40.1655,175.808 40.4362,175.808 40.707,175.808 40.9777,175.808 41.2485,175.808 
 41.5193,175.808 41.79,175.807 42.0608,175.807 42.3315,175.807 42.6023,175.807 42.873,175.807 43.1438,175.806 43.4145,175.806 43.6853,175.806 43.956,175.806 
 44.2268,175.806 44.4975,175.805 44.7683,175.805 45.039,175.805 45.3098,175.805 45.5805,

In [36]:
dataset = GenerateDatasets.MagdonIsmailAndAtiya()
data = rand(dataset)
@show loglikelihood(dataset, data)
est = NeuralDensityEstimator([100], approximate_support(dataset))


loglikelihood(dataset, data) = -772.542882959356


2017-09-12 17:13:29.527306: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1030] Creating TensorFlow device (/gpu:0) -> (device: 0, name: GeForce GTX TITAN X, pci bus id: 0000:01:00.0)


NeuralDensityEstimator(Session(Ptr{Void} @0x00007fc39c56e850), <Tensor Group:1 shape=unknown dtype=Any>, <Tensor Group_2:1 shape=unknown dtype=Any>, <Tensor t:1 shape=(1, ?) dtype=Float32>, <Tensor pdf:1 shape=unknown dtype=Float64>)

In [37]:
condition!(est)
fit!(est, data; epochs=10_000)

(ii, ysmin, ysmax, condition_loss) = (1, [1.00511], [1.00514], [0.989777])
(ii, ysmin, ysmax, condition_loss) = (51, [1.22405], [1.29118], [0.552626])
(ii, ysmin, ysmax, condition_loss) = (101, [1.25517], [1.9354], [0.069282])
(ii, ysmin, ysmax, condition_loss) = (151, [1.13798], [1.9875], [0.0191936])
(ii, ysmin, ysmax, condition_loss) = (201, [1.09839], [1.99411], [0.0097146])
(ii, ysmin, ysmax, condition_loss) = (251, [1.0776], [1.99634], [0.00603483])
(ii, ysmin, ysmax, condition_loss) = (301, [1.06446], [1.99744], [0.00416216])
(ii, ysmin, ysmax, condition_loss) = (351, [1.05532], [1.9981], [0.00306406])
(ii, ysmin, ysmax, condition_loss) = (401, [1.04855], [1.99852], [0.00235938])
(ii, ysmin, ysmax, condition_loss) = (451, [1.04332], [1.99882], [0.00187767])
(ii, ysmin, ysmax, condition_loss) = (501, [1.03914], [1.99903], [0.00153256])
(ii, ysmin, ysmax, condition_loss) = (551, [1.03571], [1.99919], [0.00127614])
(ii, ysmin, ysmax, condition_loss) = (601, [1.03286], [1.99931], [0

Epoch 6801: loss: Any[[1.0], [1.00149], [802.316], [802.416], nothing]
Epoch 6901: loss: Any[[1.0], [1.00151], [801.826], [801.925], nothing]
Epoch 7001: loss: Any[[1.0], [1.00148], [801.342], [801.442], nothing]
Epoch 7101: loss: Any[[1.0], [1.00146], [800.865], [800.965], nothing]
Epoch 7201: loss: Any[[1.0], [1.00146], [800.395], [800.494], nothing]
Epoch 7301: loss: Any[[1.0], [1.00143], [799.931], [800.03], nothing]
Epoch 7401: loss: Any[[1.0], [1.00144], [799.473], [799.573], nothing]
Epoch 7501: loss: Any[[1.0], [1.00142], [799.022], [799.122], nothing]
Epoch 7601: loss: Any[[1.0], [1.00142], [798.577], [798.677], nothing]
Epoch 7701: loss: Any[[1.0], [1.0014], [798.139], [798.238], nothing]
Epoch 7801: loss: Any[[1.0], [1.00139], [797.722], [797.822], nothing]
Epoch 7901: loss: Any[[1.0], [1.0014], [797.28], [797.38], nothing]
Epoch 8001: loss: Any[[1.0], [1.00138], [796.86], [796.959], nothing]
Epoch 8101: loss: Any[[1.0], [1.00136], [796.446], [796.545], nothing]
Epoch 8201: 

NeuralDensityEstimator(Session(Ptr{Void} @0x00007fc39c56e850), <Tensor Group:1 shape=unknown dtype=Any>, <Tensor Group_2:1 shape=unknown dtype=Any>, <Tensor t:1 shape=(1, ?) dtype=Float32>, <Tensor pdf:1 shape=unknown dtype=Float64>)

In [38]:
X=minimum(approximate_support(dataset)) : 0.01 : maximum(approximate_support(dataset))
plot(X, pdf.(est,X), xlims= approximate_support(dataset), ylims=(0,1))

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -50 
 
 
 -25 
 
 
 0 
 
 
 25 
 
 
 50 
 
 
 0.0 
 
 
 0.2 
 
 
 0.4 
 
 
 0.6 
 
 
 0.8 
 
 
 1.0 
 
<polyline clip-path="url(#clip02)" style="stroke:#009af9; stroke-width:1; stroke-opacity:1; fill:none" points="
 32.2221,375.77 32.2769,375.77 32.3317,375.77 32.3865,375.77 32.4413,375.769 32.4962,375.769 32.551,375.769 32.6058,375.769 32.6606,375.769 32.7154,375.769 
 32.7702,375.769 32.825,375.768 32.8798,375.768 32.9346,375.768 32.9894,375.768 33.0442,375.768 33.0991,375.768 33.1539,375.767 33.2087,375.767 33.2635,375.767 
 33.3183,375.767 33.3731,375.767 33.4279,375.767 33.4827,375.767 33.5375,375.766 33.5923,375.766 33.6471,375.766 33.702,375.766 33.7568,375.766 33.8116,375.766 
 33.8664,375.766 33.9212,375.765 33.976,375.765 34.0308,375.765 34.0856,375.765 34.1404,375.765 34.1952,375.765 34.25,375.764 34.3049,375.764 34.3597,375.764 
 34.4145,375.764 34.4693,375.764 34.5241,375.764 34.5789,375.763 34.6337,375.763 34.6885,375.763 34.7433,375.763 34.7981,375.763 34.853,375.763 34.9078,375.762 
 34.9626,375.762 35.0174,375.762 35.0722,375.762 35.127,375.762 35.1818,375.762 35.2366,375.762 35.2914,375.761 35.3462,375.761 35.401,375.761 35.4559,375.761 
 35.5107,375.761 35.5655,375.76 35.6203,375.76 35.6751,375.76 35.7299,375.76 35.7847,375.76 35.8395,375.76 35.8943,375.759 35.9491,375.759 36.0039,375.759 
 36.0588,375.759 36.1136,375.759 36.1684,375.759 36.2232,375.758 36.278,375.758 36.3328,375.758 36.3876,375.758 36.4424,375.758 36.4972,375.758 36.552,375.757 
 36.6068,375.757 36.6617,375.757 36.7165,375.757 36.7713,375.757 36.8261,375.756 36.8809,375.756 36.9357,375.756 36.9905,375.756 37.0453,375.756 37.1001,375.755 
 37.1549,375.755 37.2098,375.755 37.2646,375.755 37.3194,375.755 37.3742,375.755 37.429,375.754 37.4838,375.754 37.5386,375.754 37.5934,375.754 37.6482,375.754 
 37.703,375.753 37.7578,375.753 37.8127,375.753 37.8675,375.753 37.9223,375.753 37.9771,375.752 38.0319,375.752 38.0867,375.752 38.1415,375.752 38.1963,375.752 
 38.2511,375.751 38.3059,375.751 38.3607,375.751 38.4156,375.751 38.4704,375.751 38.5252,375.75 38.58,375.75 38.6348,375.75 38.6896,375.75 38.7444,375.75 
 38.7992,375.749 38.854,375.749 38.9088,375.749 38.9636,375.749 39.0185,375.749 39.0733,375.748 39.1281,375.748 39.1829,375.748 39.2377,375.748 39.2925,375.748 
 39.3473,375.747 39.4021,375.747 39.4569,375.747 39.5117,375.747 39.5665,375.746 39.6214,375.746 39.6762,375.746 39.731,375.746 39.7858,375.746 39.8406,375.745 
 39.8954,375.745 39.9502,375.745 40.005,375.745 40.0598,375.744 40.1146,375.744 40.1695,375.744 40.2243,375.744 40.2791,375.744 40.3339,375.743 40.3887,375.743 
 40.4435,375.743 40.4983,375.743 40.5531,375.742 40.6079,375.742 40.6627,375.742 40.7175,375.742 40.7724,375.742 40.8272,375.741 40.882,375.741 40.9368,375.741 
 40.9916,375.741 41.0464,375.74 41.1012,375.74 41.156,375.74 41.2108,375.74 41.2656,375.739 41.3204,375.739 41.3753,375.739 41.4301,375.739 41.4849,375.738 
 41.5397,375.738 41.5945,375.738 41.6493,375.738 41.7041,375.737 41.7589,375.737 41.8137,375.737 41.8685,375.737 41.9233,375.737 41.9782,375.736 42.033,375.736 
 42.0878,375.736 42.1426,375.736 42.1974,375.735 42.2522,375.735 42.307,375.735 42.3618,375.735 42.4166,375.734 42.4714,375.734 42.5263,375.734 42.5811,375.733 
 42.6359,375.733 42.6907,375.733 42.7455,375.733 42.8003,375.732 42.8551,375.732 42.9099,375.732 42.9647,375.732 43.0195,375.731 43.0743,375.731 43.1292,375.731 
 43.184,375.731 43.2388,375.73 43.2936,375.73 43.3484,375.73 43.4032,375.73 43.458,375.729 43.5128,375.729 43.5676,375.729 43.6224,375.728 43.6772,375.728 
 43.7321,375.728 43.7869,375.728 43.8417,375.727 43.8965,375.727 43.9513,375.727 44.0061,375.727 44.0609,375.726 44.1157,375.726 44.1705,375.726 44.2253,375.725 
 44.2801,375.725 44.335,375.725 44.3898,375.725 44.4446,375.724 44.4994,375.724 44.5542,375.724 44.609,375.723 44.6638,375.723 44.7186,375.723

In [39]:
histogram(data, nbins=50)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -25 
 
 
 0 
 
 
 25 
 
 
 0 
 
 
 3 
 
 
 6 
 
 
 9 
 
 
 12 
 
 
 15 
 
 
 18 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 y1

In [47]:
dataset = Arcsine(1,4)
data = rand(dataset, 5000)
@show loglikelihood(dataset, data)
est = NeuralDensityEstimator([100], approximate_support(dataset))


loglikelihood(dataset, data) = -4235.659430071057


2017-09-12 17:17:51.522335: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1030] Creating TensorFlow device (/gpu:0) -> (device: 0, name: GeForce GTX TITAN X, pci bus id: 0000:01:00.0)


NeuralDensityEstimator(Session(Ptr{Void} @0x00007fc39c7458f0), <Tensor Group:1 shape=unknown dtype=Any>, <Tensor Group_2:1 shape=unknown dtype=Any>, <Tensor t:1 shape=(1, ?) dtype=Float32>, <Tensor pdf:1 shape=unknown dtype=Float64>)

In [48]:
condition!(est)
fit!(est, data; epochs=10_000)

(ii, ysmin, ysmax, condition_loss) = (1, [1.00407], [1.00407], [0.991899])
(ii, ysmin, ysmax, condition_loss) = (51, [1.24801], [1.24997], [0.624061])
(ii, ysmin, ysmax, condition_loss) = (101, [1.49691], [1.5167], [0.480492])
(ii, ysmin, ysmax, condition_loss) = (151, [1.50135], [1.54809], [0.455569])
(ii, ysmin, ysmax, condition_loss) = (201, [1.49981], [1.58203], [0.424506])
(ii, ysmin, ysmax, condition_loss) = (251, [1.4952], [1.61869], [0.390623])
(ii, ysmin, ysmax, condition_loss) = (301, [1.48773], [1.65483], [0.357022])
(ii, ysmin, ysmax, condition_loss) = (351, [1.47819], [1.68796], [0.326034])
(ii, ysmin, ysmax, condition_loss) = (401, [1.46758], [1.71682], [0.298816])
(ii, ysmin, ysmax, condition_loss) = (451, [1.45664], [1.74146], [0.275368])
(ii, ysmin, ysmax, condition_loss) = (501, [1.44569], [1.76275], [0.254926])
(ii, ysmin, ysmax, condition_loss) = (551, [1.43449], [1.78194], [0.236332])
(ii, ysmin, ysmax, condition_loss) = (601, [1.42233], [1.80038], [0.218214])
(ii,

Epoch 6901: loss: Any[[10.6612], [14.764], [4664.05], [4665.01], nothing]
Epoch 7001: loss: Any[[10.6494], [14.73], [4662.24], [4663.18], nothing]
Epoch 7101: loss: Any[[10.6443], [14.702], [4660.44], [4661.37], nothing]
Epoch 7201: loss: Any[[10.6453], [14.6794], [4658.66], [4659.58], nothing]
Epoch 7301: loss: Any[[10.6511], [14.6611], [4656.91], [4657.81], nothing]
Epoch 7401: loss: Any[[10.6598], [14.645], [4655.17], [4656.06], nothing]
Epoch 7501: loss: Any[[10.6718], [14.6317], [4653.46], [4654.33], nothing]
Epoch 7601: loss: Any[[10.6864], [14.6212], [4651.76], [4652.62], nothing]
Epoch 7701: loss: Any[[10.7028], [14.6126], [4650.09], [4650.94], nothing]
Epoch 7801: loss: Any[[10.7203], [14.6045], [4648.44], [4649.27], nothing]
Epoch 7901: loss: Any[[10.7389], [14.598], [4646.81], [4647.63], nothing]
Epoch 8001: loss: Any[[10.7572], [14.5911], [4645.2], [4646.0], nothing]
Epoch 8101: loss: Any[[10.7765], [14.5855], [4643.61], [4644.4], nothing]
Epoch 8201: loss: Any[[10.7955], [

NeuralDensityEstimator(Session(Ptr{Void} @0x00007fc39c7458f0), <Tensor Group:1 shape=unknown dtype=Any>, <Tensor Group_2:1 shape=unknown dtype=Any>, <Tensor t:1 shape=(1, ?) dtype=Float32>, <Tensor pdf:1 shape=unknown dtype=Float64>)

demo_plot (generic function with 2 methods)

In [92]:
plot([data],
    xlims= approximate_support(dataset),
    seriestype = [ :histogram],
    legend=false
)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 1 
 
 
 2 
 
 
 3 
 
 
 4 
 
 
 0 
 
 
 500 
 
 
 1000

In [61]:
plot(data2, seriestype=:histogram)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -2 
 
 
 0 
 
 
 2 
 
 
 0 
 
 
 3 
 
 
 6 
 
 
 9 
 
 
 12 
 
 
 15 
 
 
 18 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 y1

In [88]:
?histogram!

search: histogram! histogram2d! histogram Histogram histogram2d



No documentation found.

`Plots.histogram!` is a `Function`.

```
# 1 method for generic function "histogram!":
histogram!(args...; kw...) in Plots at /home/uniwa/students2/students/20361362/linux/.julia/v0.6/RecipesBase/src/RecipesBase.jl:360
```
